## Scrape Data from Robert Koch Institut
https://experience.arcgis.com/experience/478220a4c454480e823b17327b2bf1d4/page/page_0/

In [ ]:
import urllib
import json
import datetime
import pandas


In [ ]:
def fetch_infection_data_from_rki(bundesland:str="Hamburg"):
    """
    Fetch Covid-19-Cases from 
    https://experience.arcgis.com/experience/478220a4c454480e823b17327b2bf1d4/page/page_0/
    
    Args:
        bundesland: written like displayed on the website, a string
    Returns:
        a Dataframe containing all historical data from a bundesland
    """
    
    url_endpoint = "https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_COVID19/FeatureServer/0/query"
    params = {
        'f': 'json', 
        'where': f'Bundesland=\'{bundesland}\'',
        'returnGeometry': 'false',
        'spatialRel': 'esriSpatialRelIntersects',
        'outFields': 'ObjectId,AnzahlFall,Meldedatum',
        'orderByFields': 'Meldedatum asc',
        'resultOffset': 0,
        'resultRecordCount': 2000,
        'cacheHint': "true"    
    }

    url_query = f"{url_endpoint}?{urllib.parse.urlencode(params)}"

    with urllib.request.urlopen(url_query) as url:
        data = json.loads(url.read().decode())['features']
        print(data[1])
    
    data_list = [
        (datetime.datetime.fromtimestamp(x['attributes']['Meldedatum'] / 1e3), x['attributes']['AnzahlFall'],bundesland) 
        for x in data
    ]

    df = pandas.DataFrame(data_list, columns=['Meldedatum', 'AnzahlFall', 'Bundesland'])

    return df

In [ ]:
df = fetch_infection_data_from_rki("Bayern")
df.head()